# 0.0a Data Preparation
Garrett Eason, Chris Broll, Shilpa Rajbhandari

In [ ]:
# Packages 
import pandas as pd
!pip install openpyxl
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import datetime as dt

#%% Locations
Windows = 'C:/Users/Sade/Documents/GitHub/lights-and-crime/Lights and Crime Garrett/Data'
Linux = '/home/sade/Desktop/Git Cloned Repos/lights-and-crime/Lights and Crime Garrett/Data'

choice = Linux

## Crime Data
Acquired from:<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2017<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2016<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2015<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2014<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2013<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2012<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2011<br>
http://opendata.dc.gov/datasets/crime-incidents-in-2010<br>

### Joining Crime Data

In [ ]:
df17 = pd.read_csv('Crime_Incidents_in_2017.csv')
df16 = pd.read_csv('Crime_Incidents_in_2016.csv')
df15 = pd.read_csv('Crime_Incidents_in_2015.csv')
df14 = pd.read_csv('Crime_Incidents_in_2014.csv')
df13 = pd.read_csv('Crime_Incidents_in_2013.csv')
df12 = pd.read_csv('Crime_Incidents_in_2012.csv')
df11 = pd.read_csv('Crime_Incidents_in_2011.csv')
df10 = pd.read_csv('Crime_Incidents_in_2010.csv')

crimes = pd.concat([df17, df16, df15, df14, df13, df12, df11, df10], axis=0, join='inner')

## iSlims and City Work Data
Acquired from:<br>
https://github.com/thelabdc/lights-and-crime/tree/master/data<br>
http://opendata.dc.gov/datasets/cityworks-workorders<br>

*Note: Be careful about citywork data and Github, the data set is too lage to upload all at once.

### Setting up iSlims and City Work Data

In [ ]:
wo = pd.read_excel(choice + '/islims_workorders.xlsx')
de = pd.read_excel(choice + '/islims_workorders_detail.xlsx')
iv = pd.read_excel(choice + '/islims_inventory.xlsx')
fc = pd.read_excel(choice + '/islims_failure_codes.xlsx')
CW = pd.read_csv(choice + '/Cityworks_Workorders.csv') # Will need to add this dataset (too large for GitHub)

#%% Merging and Cleaning Data

# Merging work order and details sets on woID
wode = pd.merge(wo, de, on='woID')

# Merging wode and inventory sets on inventoryID
iSlimsa = pd.merge(wode, iv, on='inventoryID')

# Merging the GPS coordinates to fill in as many NaN gaps as possible
gpsX = iSlimsa['gpscoordinateX_x'].combine_first(iSlimsa['gpscoordinateX_y'])
gpsX = gpsX.combine_first(iSlimsa['gpscoordinateX_x'])
gpsY = iSlimsa['gpscoordinateY_x'].combine_first(iSlimsa['gpscoordinateY_y'])
gpsY = gpsY.combine_first(iSlimsa['gpscoordinateY_x'])

# Removing irrelevant variables
iSlimsb = iSlimsa.iloc[:,0:18]
iSlimsb['gpsX'] = gpsX
iSlimsb['gpsY'] = gpsY

# final merged dataset
iSlimsc = iSlimsb.drop(['gpscoordinateX_x', 'gpscoordinateY_x'], axis = 1)

# Throwing out observations without GPS coordinate
iSlimsd = iSlimsc.dropna(subset = ['gpsX', 'gpsY'])

# Will use codes: 2, 196, 201, 209
fc[fc['description'].str.contains('ight')].head()

# Filtering by observations with the desired failure codes
iSlimse = iSlimsd[iSlimsd['finalresolutionID'].isin([2, 196, 201, 209])]

# Filtering by times of interest
resolv_t = (iSlimse['resolveddatetime'] > '2007-12-31') & (iSlimse['resolveddatetime'] < '2017-01-01') 
enter_t = (iSlimse['entereddate_x'] > '2007-12-31') & (iSlimse['entereddate_x'] < '2017-01-01')
iSlimsf = iSlimse[resolv_t & enter_t]

# Filtering out observations with excessively long completion / late completion times
iSlimsg = iSlimsf[(iSlimsf['daysToComplete'] <= 23)]
iSlimsg['finalresolutionID'].value_counts()

# Dropping duplicate woID's
iSlimsh = iSlimsg.drop_duplicates(subset = ['woID'])

# Throwing out observations with a GPS coordinate that is too large in magnitude to be possible
iSlimsh[['gpsX', 'gpsY']] = iSlimsh[['gpsX', 'gpsY']].apply(pd.to_numeric)
iSlims = iSlimsh[iSlimsh['gpsX'] <= 20000 ]
iSlims = iSlims[iSlims['gpsY']<= 20000]

# Removing 2 observations whos GPS coordinates were entered incorrectly and limiting bounds of GPS coordinates to realistic numbers in the bounds of DC
iSlims = iSlims.drop([465076, 144970])
iSlims = iSlims[(iSlims['gpsX'] >= 38.7) & (iSlims['gpsX'] <= 39) ]
iSlims = iSlims[(iSlims['gpsY'] >= -77.15) & (iSlims['gpsY'] <= -76.90)]
# Cutting out close outliers
iSlims = iSlims[~((iSlims['gpsX'] >= 38.828) & (iSlims['gpsX'] <= 38.8395) & (iSlims['gpsY'] <= -76.9632) & (iSlims['gpsY'] >= -76.9777))]
iSlims = iSlims[~((iSlims['gpsX'] >= 38.955) & (iSlims['gpsX'] <= 38.96) & (iSlims['gpsY'] >= -76.98) & (iSlims['gpsY'] <= -76.97))]
iSlims = iSlims.reset_index()
del iSlims['index']

# For some reason the coordinate system is backwards
iSlims[['gpsX', 'gpsY']] = iSlims[['gpsY', 'gpsX']]

#%% Merged Ready and Standardized iSlims Data

# Standardized iSlims
SDiSlims = iSlims[['woID', 'entereddate_x', 'resolveddatetime', 'gpsX', 'gpsY']]
SDiSlims.columns = ['WoID', 'WoEntered', 'WoCompleted', 'gpsX', 'gpsY']

# Readying City Works Data
some_vars = ["WORKORDERID","WORKORDERCLOSEDDATE","INITIATEDDATE","DESCRIPTION", "LATITUDE", "LONGITUDE", "X", "Y"]
CW_1 = CW[CW["DESCRIPTION"]=="LIGHT POLE LIGHT OUT"][some_vars]
CW_2 = CW[CW["DESCRIPTION"]=="LIGHT MALFUNCTION"][some_vars]
CW_new = pd.concat([CW_1,CW_2])
CW_new = CW_new.sort_values(by="INITIATEDDATE", ascending=False)
CW_new = CW_new.reset_index(drop = True)
final_vars = ["WORKORDERID","INITIATEDDATE", "WORKORDERCLOSEDDATE","X", "Y"]
CW_new = CW_new[final_vars]
CW_new.columns = ["WoID", "WoEntered", "WoCompleted", "gpsX", "gpsY"]
CW_new = CW_new[(CW_new['gpsY'] >= 38.7)]

#%% Merging iSlims and Citywork
Lights = pd.concat([SDiSlims, CW_new], ignore_index = True)

### Separating Out Night Crimes
To avoid exogeneity problems due to sun rise/set times, we will define the following:<br>
Night will be defined as the shortest time difference between nautical twilight (21:00 - 5:00 Military time);<br>
Day will be defined as the shortest time difference between civil twilight (7:00 - 17:00).

In [10]:
crimes['REPORT_DAT'] = [dt.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%fZ') for date in crimes['REPORT_DAT']]
NCR = crimes[(crimes.REPORT_DAT.dt.hour <= 5) | (crimes['REPORT_DAT'].dt.hour > 21)]
DCR = crimes[(crimes.REPORT_DAT.dt.hour <= 17) & (crimes['REPORT_DAT'].dt.hour > 7)]

## Writing Data

In [11]:
crimes.to_excel(choice + '/crimes.xlsx')
SDiSlims.to_excel(choice + '/SDiSlims_final.xlsx')
NCR.to_excel(choice + '/NCR.xlsx')
DCR.to_excel(choice + '/DCR.xlsx')
Lights.to_excel(choice + '/Lights.xlsx')